# Baseline I: SARIMA


In [2]:
from datetime import timedelta
import itertools
import json
import numpy as np
import os
import pandas as pd
from pathlib import Path
import yaml
import mlflow
from datetime import datetime

# Get the current project path (where you open the notebook)
# and go up two levels to get the project path
current_dir = Path.cwd()
proj_path = current_dir.parent.parent

# make the code in src available to import in this notebook
import sys
sys.path.append(os.path.join(proj_path, 'src'))

# Custom functions and classes
from sarima import SklearnSarima
from utils import make_dates, create_folder
from metrics import get_metrics

# Catalog contains all the paths related to datasets
with open(os.path.join(proj_path, 'conf/catalog.yml'), "r") as f:
    catalog = yaml.safe_load(f)['breakfast']
    
# Params contains all of the dataset creation parameters and model parameters
with open(os.path.join(proj_path, 'conf/params.yml'), "r") as f:
    params = yaml.safe_load(f)

In [3]:
# Step 1: Load the data, convert to a proper datetime format and apply correction
merged_data = pd.read_csv(os.path.join(proj_path,
                                       catalog['output_dir']['dir'], 
                                       catalog['output_dir']['merged']))
merged_data['WEEK_END_DATE'] = pd.to_datetime(merged_data['WEEK_END_DATE'])
merged_data['WEEK_END_DATE'] = merged_data['WEEK_END_DATE'] + timedelta(days=3)

# Step 2: Create date folds
date_ranges = make_dates(params['breakfast']['experiment_dates'])

In [3]:
# We are predicting on Saturday

# Fold 1:

# validat Saturday 2010-12-11	  -> Last Google Trends hits covers Sunday 2010-11-28 to Saturday 2010-12-04
#                                    Thus we add + 6 days, so that the date for that period will be labelled
#                                    2010-12-11 and will be used to predict 2010-12-11.
#                                 -> For Breakfast at the Frat, we need to add + 3 days so the days match
#                                    and corresponds to end of week. The sales from previous week can only be
#                                    used. 

# validat Saturday 2010-12-18	
# validat Saturday 2010-12-25
# validat Saturday 2011-01-01	

# predict Saturday 2011-01-08
# predict Saturday 2011-01-15
# predict Saturday 2011-01-22
# predict Saturday 2011-01-29

# Fold 2:

# validat Saturday 2011-01-08
# validat Saturday 2011-01-15
# validat Saturday 2011-01-22
# validat Saturday 2011-01-29

# predict Saturday 2011-02-05
# predict Saturday 2011-02-12
# predict Saturday 2011-02-19
# predict Saturday 2011-02-26

In [5]:
make_dates(params['breakfast']['experiment_dates'])

,train_start,train_end,valid_start,valid_end,test_start,test_end
0,2009-01-17,2010-12-04,2010-12-11,2011-01-01,2011-01-08,2011-01-29
1,2009-02-14,2011-01-01,2011-01-08,2011-01-29,2011-02-05,2011-02-26
2,2009-03-14,2011-01-29,2011-02-05,2011-02-26,2011-03-05,2011-03-26
3,2009-04-11,2011-02-26,2011-03-05,2011-03-26,2011-04-02,2011-04-23
4,2009-05-09,2011-03-26,2011-04-02,2011-04-23,2011-04-30,2011-05-21
5,2009-06-06,2011-04-23,2011-04-30,2011-05-21,2011-05-28,2011-06-18
6,2009-07-04,2011-05-21,2011-05-28,2011-06-18,2011-06-25,2011-07-16
7,2009-08-01,2011-06-18,2011-06-25,2011-07-16,2011-07-23,2011-08-13
8,2009-08-29,2011-07-16,2011-07-23,2011-08-13,2011-08-20,2011-09-10
9,2009-09-26,2011-08-13,2011-08-20,2011-09-10,2011-09-17,2011-10-08


In [6]:
from pprint import pprint

In [7]:
pprint(params['breakfast']['dataset'])

{'store_ids': {389: 'KY', 2277: 'OH', 25229: 'TX'},
 'upc_ids': {1111009477: 'bag_snacks',
             1600027527: 'cold_cereal',
             3800031838: 'cold_cereal',
             7192100339: 'frozen_pizza'}}


In [8]:
list(params['breakfast']['dataset']['store_ids'].keys())

[2277, 389, 25229]

# Experiment


In [10]:
# Setup MLFLOW
# One experiment will be store_id + upc_id
# Initialize experiment logging location
# Create mlflow tracking folder
create_folder(os.path.join(proj_path, 'mlruns'))

# Step 1: Load the data, convert to a proper datetime format and apply correction
merged_data = pd.read_csv(os.path.join(proj_path,
                                       catalog['output_dir']['dir'],  
                                       catalog['output_dir']['merged']))
merged_data['WEEK_END_DATE'] = pd.to_datetime(merged_data['WEEK_END_DATE'])
merged_data['WEEK_END_DATE'] = merged_data['WEEK_END_DATE'] + timedelta(days=3)

# Step 2: Create date folds
date_ranges = make_dates(params['breakfast']['experiment_dates'])

# Step 3: Iterate over each store and upc pair.
# For each pair, iterate over each period (fold), find the optimal
# set of parameters for that fold and make the predictions
# on the test period
stores = list(params['breakfast']['dataset']['store_ids'].keys())
upcs = list(params['breakfast']['dataset']['upc_ids'].keys())
store_upc_pairs = list(itertools.product(stores, upcs))

for store_id, upc_id in store_upc_pairs: 
    print(f'Processing store {store_id} upc {upc_id}')
    mlflow.set_tracking_uri(os.path.join(proj_path, 'mlruns'))
    mlflow.set_experiment(f'{store_id}_{upc_id}')
    
    # Iterate over each period, unpack tuple in each variable.
    # in each of the period, we will find the best set of parameters,
    # which will represent the time-series cross validation methodology
    for _, train_start, train_end, valid_start, valid_end, test_start, test_end in date_ranges.itertuples():
        
        # Define set of parameters for SARIMA
        p = d = q = range(0, 2)
        pdq = list(itertools.product(p, d, q))
        spdq = list(itertools.product(p, d, q, [2,3,4]))
        all_params = list(itertools.product(pdq, spdq))
    
        # Step 4: Filter data for the specific store and upc pair and dates.
        # The dates are inclusive
        train_x = merged_data[(merged_data['WEEK_END_DATE']>=train_start) &
                              (merged_data['WEEK_END_DATE']<=train_end) &
                              (merged_data['STORE_NUM']==store_id) &
                              (merged_data['UPC']==upc_id)]['UNITS']
        valid_y = merged_data[(merged_data['WEEK_END_DATE']>=valid_start) &
                              (merged_data['WEEK_END_DATE']<=valid_end) &
                              (merged_data['STORE_NUM']==store_id) &
                              (merged_data['UPC']==upc_id)]['UNITS']
        test_y = merged_data[(merged_data['WEEK_END_DATE']>=test_start) &
                             (merged_data['WEEK_END_DATE']<=test_end) &
                             (merged_data['STORE_NUM']==store_id) &
                             (merged_data['UPC']==upc_id)]['UNITS']
    
        # Step 5: Initiate the model with the training data to use to fit
        # Then use the fit_best_params that will iterate over all parameter combinations
        # and evaluate which performs the best on the valid data using MAPE to
        # evaluate the predictions. Then it will select the parameters that minimized
        # this metric and fit using the training and validation data.
        model = SklearnSarima(train_x.values)
        model.fit_best_params(valid_y.values, all_params)
    
        # Step 6: Make the predictions. There are two options, a: forecast for four days
        # and b: forecast for one day, and always use the most relevant data to re-fit
        # the model using the same parameters found using the fit_best_params method.
        # The prefix lt is used to indicate option a and nd indicates option b
        # For option a, we only need to provide how many steps to forecast, while option
        # b needs the actual test values, to be able to re-fit every day
        lt_predictions = model.predict(test_y.values.size)
        nd_predictions = model.fit_predict(test_y.values)
    
        # Step 7: Calculate the metrics for both forecasting techniques.
        # metrics used are: MSE, MAPE, R2, WAPE and RMSE
        lt_metrics = get_metrics(test_y.values, lt_predictions)
        nd_metrics = get_metrics(test_y.values, nd_predictions)
        used_params = model.get_params()
    
        # Step 8: Store the actual predictions in the respective folder.
        # Also ensure that the folder exists by calling create_folder. If it does it will
        # not do anything.
        fdir = os.path.join(proj_path, catalog['results']['dir'], f'{str(test_end.date())}')
        fname = os.path.join(fdir, f'sarima_{store_id}_{upc_id}.csv')
        create_folder(fdir)

        save_data = pd.DataFrame({'y_true':test_y.values,
                                  'y_pred_lt':lt_predictions,
                                  'y_pred_nd':np.array(nd_predictions).flatten(),
                                  'dates':merged_data[(merged_data['WEEK_END_DATE']>=test_start) &
                                                     (merged_data['WEEK_END_DATE']<=test_end) &
                                                     (merged_data['STORE_NUM']==store_id) &
                                                     (merged_data['UPC']==upc_id)]['WEEK_END_DATE'].values})
        save_data.to_csv(fname)
        with mlflow.start_run():
            mlflow.log_artifact(fname)
            mlflow.log_params(used_params)
            mlflow.log_metrics(nd_metrics)


Finding best parameters:   2%|▏         | 4/192 [00:00<00:05, 32.07it/s]

Processing store 2277 upc 1600027527
INFO: '2277_1600027527' does not exist. Creating a new experiment


Finding best parameters: 100%|██████████| 192/192 [00:15<00:00, 12.17it/s]
